In [8]:
from hdmm import workload, templates, error
import pickle
import numpy as np
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
W = pickle.load(open('household_wkld.pckl','rb'))

dom = workload.Marginals.approximate(W).domain

temp = templates.Marginals(dom)
A, loss = temp.restart_optimize(W, 5)
print('RMSE', np.sqrt(loss / W.shape[0]))

RMSE 20.221280824930187


In [37]:
# this takes a while
error_matrix = np.zeros((len(W.matrices), len(A.matrices)))
for i, Wi in enumerate(W.matrices):
    #print(i)
    for j, Aj in enumerate(A.matrices):
        error_matrix[i,j] = error.expected_error(Wi, Aj.base)
    

In [38]:
best_strategy = np.argmin(error_matrix, axis=1)
groups = [ [] for i in range(len(A.matrices)) ]
for i in range(len(W.matrices)):
    j = best_strategy[i]
    groups[j].append(W.matrices[i])
    
groups = [workload.VStack(g) for g in groups if len(g) > 0]

In [44]:
opt_plus = templates.DefaultUnionKron(dom, len(groups))
B, loss = opt_plus.restart_optimize(groups, 5)

print('RMSE', np.sqrt(loss / W.shape[0]))

RMSE 23.016444551567005


In [43]:
groups

[<21x387072 VStack with dtype=bool>,
 <9x387072 VStack with dtype=bool>,
 <156x387072 VStack with dtype=bool>,
 <6x387072 VStack with dtype=bool>,
 <22x387072 VStack with dtype=bool>,
 <16x387072 VStack with dtype=bool>,
 <161x387072 VStack with dtype=bool>,
 <97x387072 VStack with dtype=bool>,
 <20x387072 VStack with dtype=bool>,
 <46x387072 VStack with dtype=bool>,
 <26x387072 VStack with dtype=bool>]